# Automatización de datos de Banxico


Para utilizar la API de series de tiempo de Banxico, es necesario primero **generar un token de consulta**, el cual se puede obtener en la siguiente [liga](https://www.banxico.org.mx/SieAPIRest/service/v1/token). 

Este token se genera una sola vez y se usa cuantas veces se necesite, respetando los límites de consulta (10,000 consultas al día y/o 200 consultas en 5 minutos por token). 

Se pueden descargar varias series a la vez en la misma función, siempre que sean de la misma periodicidad

In [ ]:
#series de prueba: TIIE 28: SF43783; inflación anual: SP30578; salario mínimo índice real: SL11297
#probar con varias series (mensuales, tienen que tener la misma periodicidad):SF283 (TIIE 28), SP30578 (inflación),SL11297 (salario mínimo)

### Función para descargar series de Banxico

In [30]:
import requests
import numpy as np
import pandas as pd

#pegar aquí el token que se generó en la liga anterior
token_1='82908162e488cf6a1e53620862cd2d9fa1181ba5a78882ffeb5d29320a19a145'

def descargar_banxico(serie,fecha_inicio,fecha_fin,token,es_oportuno=False):
    #Donde: 
    # serie: el identificador que tiene Banxico para cada serie de tiempo en su sistema de información
    # fecha_inicio: fecha en la que se quiere iniciar la consulta
    # fecha_fin: fecha en la que se quiere concluir la consulta
    # token: el token generado por la API de Banxico para acceder

    if es_oportuno==True:
        url='https://www.banxico.org.mx/SieAPIRest/service/v1/series/'+serie+'/datos/oportuno?token='+token
    else:
        url='https://www.banxico.org.mx/SieAPIRest/service/v1/series/'+serie+'/datos/'+fecha_inicio+'/'+fecha_fin+'?token='+token
    headers = {'Token':token}
    response = requests.get(url,headers=headers)
    status=response.status_code
    raw_data = response.json()
    T=np.shape(serie.split(","))[0]
    info={'fecha':pd.DataFrame(raw_data['bmx']['series'][0]['datos'])['fecha']}

    for i in range(T):
        titulo=raw_data['bmx']['series'][i]['titulo']
        
        data=raw_data['bmx']['series'][i]['datos']
        
        df=pd.DataFrame(data)
        df['dato'] = df['dato'].apply(lambda x:float(x))
        info[titulo]=df['dato']
    info=pd.DataFrame(info)
    info.set_index('fecha',inplace=True)

    return info

descargar_banxico('SP30578,SL11297','2016-01-01','2023-05-01',token_1)


,"Salarios Mínimos General Índice Real, Dic2018=100",Índice Nacional de Precios al consumidor variación anual
fecha,,
01/01/2016,95.2753,2.61
01/02/2016,94.8599,2.87
01/03/2016,94.7204,2.60
01/04/2016,95.0213,2.54
01/05/2016,95.4469,2.60
...,...,...
01/01/2023,189.9359,7.91
01/02/2023,188.8828,7.62
01/03/2023,188.3781,6.85


### Pruebas de la función de descarga

In [44]:
descargar_banxico('SP74665,SF61745,SF60634,SF43773','2016-01-01','2023-05-01',token_1)

{'bmx': {'series': [{'idSerie': 'SF60634',
    'titulo': 'Valores gubernamentales, Resultados de la subasta semanal Cetes a 91 días - Tasa de rendimiento - Fecha subasta',
    'datos': [{'fecha': '05/01/2016', 'dato': '3.2700'},
     {'fecha': '12/01/2016', 'dato': '3.2800'},
     {'fecha': '19/01/2016', 'dato': '3.2900'},
     {'fecha': '26/01/2016', 'dato': '3.3400'},
     {'fecha': '02/02/2016', 'dato': '3.3600'},
     {'fecha': '09/02/2016', 'dato': '3.3800'},
     {'fecha': '16/02/2016', 'dato': '3.4000'},
     {'fecha': '23/02/2016', 'dato': '3.9900'},
     {'fecha': '01/03/2016', 'dato': '3.9800'},
     {'fecha': '08/03/2016', 'dato': '3.9300'},
     {'fecha': '15/03/2016', 'dato': '3.9100'},
     {'fecha': '18/03/2016', 'dato': '3.8700'},
     {'fecha': '29/03/2016', 'dato': '3.8700'},
     {'fecha': '05/04/2016', 'dato': '3.8600'},
     {'fecha': '12/04/2016', 'dato': '3.8400'},
     {'fecha': '19/04/2016', 'dato': '3.7900'},
     {'fecha': '26/04/2016', 'dato': '3.8300'},
   

In [18]:
url1='https://www.banxico.org.mx/SieAPIRest/service/v1/series/SP74665,SF61745,SF60634,SF43773/datos/2016-01-01/2023-01-01?token='+token_1
headers = {'Token':token_1}
response = requests.get(url1,headers=headers)
status=response.status_code
raw_data = response.json()
raw_data

{'bmx': {'series': [{'idSerie': 'SP74665',
    'titulo': 'Inflación No subyacente (nueva definición) Anual',
    'datos': [{'fecha': '01/01/2016', 'dato': '2.5200'},
     {'fecha': '01/02/2016', 'dato': '3.4900'},
     {'fecha': '01/03/2016', 'dato': '2.1200'},
     {'fecha': '01/04/2016', 'dato': '1.6600'},
     {'fecha': '01/05/2016', 'dato': '1.5500'},
     {'fecha': '01/06/2016', 'dato': '1.1600'},
     {'fecha': '01/07/2016', 'dato': '1.6500'},
     {'fecha': '01/08/2016', 'dato': '1.9900'},
     {'fecha': '01/09/2016', 'dato': '2.6500'},
     {'fecha': '01/10/2016', 'dato': '2.9500'},
     {'fecha': '01/11/2016', 'dato': '3.3400'},
     {'fecha': '01/12/2016', 'dato': '3.1300'},
     {'fecha': '01/01/2017', 'dato': '7.4000'},
     {'fecha': '01/02/2017', 'dato': '6.7100'},
     {'fecha': '01/03/2017', 'dato': '8.0200'},
     {'fecha': '01/04/2017', 'dato': '9.2500'},
     {'fecha': '01/05/2017', 'dato': '10.6000'},
     {'fecha': '01/06/2017', 'dato': '11.0900'},
     {'fecha': '

In [33]:
descargar_banxico('SL11297','2016-01-01','2023-05-01',token_1)


,dato
fecha,
2016-01-01,95.2753
2016-02-01,94.8599
2016-03-01,94.7204
2016-04-01,95.0213
2016-05-01,95.4469
...,...
2023-01-01,189.9359
2023-02-01,188.8828
2023-03-01,188.3781
